# Mapping accidents

This Notebook will take you through the basics of geospatial querying and plotting data on a map.

First, some boilerplate imports.

In [1]:
# Import the required libraries and open the connection to Mongo

import collections
import datetime
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (15, 15) # Reset the base size of figures so they're large enough to be useful.

import pandas as pd
import scipy.stats

import folium

import pymongo
from bson.objectid import ObjectId
from bson.son import SON

In [2]:
# Open a connection to the Mongo server, open the accidents database and name the collections of accidents and labels
client = pymongo.MongoClient('mongodb://localhost:27351/')

db = client.accidents
accidents = db.accidents
labels = db.labels
roads = db.roads

In [3]:
# Load the expanded names of keys and human-readable codes into memory
expanded_name = collections.defaultdict(str)
for e in labels.find({'expanded': {"$exists": True}}):
    expanded_name[e['label']] = e['expanded']
    
label_of = collections.defaultdict(str)
for l in labels.find({'codes': {"$exists": True}}):
    for c in l['codes']:
        try:
            label_of[l['label'], int(c)] = l['codes'][c]
        except ValueError: 
            label_of[l['label'], c] = l['codes'][c]

## Plotting some accidents

To start with, let's just plot some accidents on the map to see how it's done. Depending on which thousand accidents MongoDB picks, you may need to move and zoom the map to see the points.

The map generation part of this should be familiar to you from Part 5. The only difference is the use of the `uuid` methods to generate a unique name for each map, to save thinking of a unique name for each map.

In [4]:
m = folium.Map(location=[55, -3], width=500, height=800, zoom_start=6)

for a in accidents.find({}, ['loc.coordinates'], limit=1000):
    folium.RegularPolygonMarker(location=[a['loc']['coordinates'][1], a['loc']['coordinates'][0]], 
                     number_of_sides=5, radius=5, rotation=54).add_to(m)
m

Let's see if we can plot the UK motorway network by where the accidents are. `Road_Class` tells us they type of road the accident was on.

In [5]:
[(c, l, label_of[l, c]) for l, c in label_of  if "Road_Class" in l ]

[(4, '1st_Road_Class', 'B'),
 (5, '2nd_Road_Class', 'C'),
 (0, '2nd_Road_Class', 'Not at junction or within 20 metres'),
 (2, '1st_Road_Class', 'A(M)'),
 (2, '2nd_Road_Class', 'A(M)'),
 (3, '1st_Road_Class', 'A'),
 (1, '1st_Road_Class', 'Motorway'),
 (4, '2nd_Road_Class', 'B'),
 (5, '1st_Road_Class', 'C'),
 (6, '2nd_Road_Class', 'Unclassified'),
 (1, '2nd_Road_Class', 'Motorway'),
 (3, '2nd_Road_Class', 'A'),
 (6, '1st_Road_Class', 'Unclassified')]

This leaves us with a decision: to A(M) roads count as motorways? Let's say yes, and include them in the query. 

But before we do that, how many accidents are there?

In [6]:
accidents.find({'$or': [{'1st_Road_Class': 1}, 
                                 {'2nd_Road_Class': 1},
                                 {'1st_Road_Class': 2}, 
                                 {'2nd_Road_Class': 2}]}).count()

6274

That's quite a lot. We don't need that many to show the idea. We can restrict the subset by date, picking just the accidents in January 2012. How many are those are there?

In [7]:
accidents.find({'$or': [{'1st_Road_Class': 1}, 
                                 {'2nd_Road_Class': 1},
                                 {'1st_Road_Class': 2}, 
                                 {'2nd_Road_Class': 2}],
               'Datetime': {'$lte': datetime.datetime(2012, 1, 31)}}).count()

448

A better number. Let's plot them.

In [8]:
m = folium.Map(location=[55, -3], width=500, height=800, zoom_start=6)

for a in accidents.find({'$or': [{'1st_Road_Class': 1}, 
                                 {'2nd_Road_Class': 1},
                                 {'1st_Road_Class': 2}, 
                                 {'2nd_Road_Class': 2}],
               'Datetime': {'$lte': datetime.datetime(2012, 1, 31)}}, 
                        ['loc.coordinates']):
    folium.RegularPolygonMarker(location=[a['loc']['coordinates'][1], a['loc']['coordinates'][0]], 
                     number_of_sides=5, radius=5, rotation=54).add_to(m)
m

## Making a function
There's a lot of boilerplate to do with adding markers. Let's wrap it up in a function. 

(I've also added some other parameters we'll use later.)

In [ ]:
# def make_map(location, width=800, height=800, zoom_start=9):
#     m = folium.Map(location=location, width=width, height=height, zoom_start=zoom_start)
#     m.save('folium-map-' + uuid.uuid4().hex + '.html')
#     m.render_iframe = True
#     return m

In [9]:
def add_accidents_markers(the_map, query, limit=0,
                          number_of_sides=5, fill_color='#769d96', 
                          radius=5, rotation=54):
    for a in accidents.find(query,
                            ['loc.coordinates'],
                            limit=limit):
        folium.RegularPolygonMarker(location=[a['loc']['coordinates'][1], a['loc']['coordinates'][0]], 
                     number_of_sides=number_of_sides, radius=radius, rotation=rotation,
                                   fill_color=fill_color).add_to(the_map)    

What patch is covered by the North Wales police force (code 60)?

(Note that `zip(*list_of_lists)` is the Python idiom for "transpose", similar to the _pandas_ `.T` method.)

In [10]:
list(zip(*[[0, 1, 2], [10, 11, 12], [100, 101, 102], [1000, 1001, 1002]]))

[(0, 10, 100, 1000), (1, 11, 101, 1001), (2, 12, 102, 1002)]

In [11]:
latlons = list(zip(*[(a['loc']['coordinates'][1], 
                      a['loc']['coordinates'][0]) 
                    for a in accidents.find({'Police_Force': 60}, 
                        ['loc.coordinates'])]))
max_lat = max(latlons[0])
max_lon = max(latlons[1])
min_lat = min(latlons[0])
min_lon = min(latlons[1])
    
m = folium.Map([min_lat + (max_lat - min_lat) / 2, 
                        min_lon + (max_lon - min_lon) / 2], 
               zoom_start=9)    

add_accidents_markers(m, {'Police_Force': 60})
m

Again, a lot of boilerplate. Let's wrap the code to find the centre of a map into a function as well.

In [12]:
def map_centre_from_points(query):
    latlons = list(zip(*[(a['loc']['coordinates'][1], 
                      a['loc']['coordinates'][0]) 
                    for a in accidents.find(query, 
                        ['loc.coordinates'])]))
    max_lat = max(latlons[0])
    max_lon = max(latlons[1])
    min_lat = min(latlons[0])
    min_lon = min(latlons[1]) 
    return [min_lat + (max_lat - min_lat) / 2,
            min_lon + (max_lon - min_lon) / 2]

In [13]:
query = {'Police_Force': 60}

map_centre = map_centre_from_points(query)
  
m = folium.Map(map_centre, zoom_start=9)
add_accidents_markers(m, query)
m

### Activity 1
Plot the locations of all fatal and serious accidents that occurred on motorways.

The solution is in the [`15.1solutions`](15.1solutions.ipynb) Notebook.

In [ ]:
# Insert your solution here.
m = folium.Map(location=[55, -3], width=500, height=800, zoom_start=6)

for a in accidents.find({'$or': [{'1st_Road_Class': 1}, 
                                 {'2nd_Road_Class': 1},
               'Datetime': {'$lte': datetime.datetime(2012, 1, 31)}},
                                 ''
                        ['loc.coordinates']):
    folium.RegularPolygonMarker(location=[a['loc']['coordinates'][1], a['loc']['coordinates'][0]], 
                     number_of_sides=5, radius=5, rotation=54).add_to(m)
m

### Activity 2
Plot the accidents covered by the Cornwall local authority. Recall that local authorities are listed as `Local_Authority_(Highway)` in the dataset.

The solution is in the [`15.1solutions`](15.1solutions.ipynb) Notebook.

In [ ]:
# Insert your solution here.

## Milton Keynes
Let's zoom in a bit on Milton Keynes (MK), the home of the Open University. Let's start by just plotting all the accidents that occurred near MK.

In [ ]:
region = {'loc': 
          {'$nearSphere':
           {'$geometry': 
            {'type': 'Point', 
             'coordinates': [-0.7601851, 52.0395099]},
            '$maxDistance': 10000}}}

map_centre = map_centre_from_points(region)

m = folium.Map(map_centre, zoom_start=12)    
add_accidents_markers(m, region)
m

This bit of JSON defines the bounding rectangle around Milton Keynes. We'll use it to filter the set of accidents more closely.

In [ ]:
milton_keynes = {'type': 'Polygon',
                 'coordinates': [[[-0.877025, 52.092317],
                                  [-0.651709, 52.092317],
                                  [-0.651709, 51.958628],
                                  [-0.877025, 51.958268],
                                  [-0.877025, 52.092317]
                                 ]]}

min_mk_lat = min(p[1] for p in milton_keynes['coordinates'][0])
max_mk_lat = max(p[1] for p in milton_keynes['coordinates'][0])
min_mk_lon = min(p[0] for p in milton_keynes['coordinates'][0])
max_mk_lon = max(p[0] for p in milton_keynes['coordinates'][0])

mk_centre = [min_mk_lat + (max_mk_lat - min_mk_lat) / 2, min_mk_lon + (max_mk_lon - min_mk_lon) / 2]

mk_region_query = {'loc': {'$geoWithin': {'$geometry': milton_keynes}}}

In [ ]:
m = folium.Map(mk_centre, zoom_start=12)    
add_accidents_markers(m, mk_region_query)
m

Let's plot the different accident severities in different colours.

First, a function to merge the additional query terms for the different types into the existing location-specifying query.

In [ ]:
def merge_dicts(this, other):
    this_copy = this.copy()
    for k in other:
        this_copy[k] = other[k]
    return this_copy

# Use like:
a_dict = {'k1': 1, 'k2': 2}
merge_dicts(a_dict, {'add1': 100})

In [ ]:
m = folium.Map(mk_centre, zoom_start=12) 

add_accidents_markers(m, merge_dicts(mk_region_query, {'Accident_Severity': 1}),
    fill_color='#ff0000', number_of_sides=5, radius=10, rotation=54)

add_accidents_markers(m, merge_dicts(mk_region_query, {'Accident_Severity': 2}),
    fill_color='#ffff00', number_of_sides=4, radius=7, rotation=0)

add_accidents_markers(m, merge_dicts(mk_region_query, {'Accident_Severity': 3}),
    fill_color='#00ff00', number_of_sides=3, radius=5, rotation=30)
m

### Activity 3 
Plot the accidents around your home or workplace.

Do the accidents occur where you would expect?

Note the overly complex structure of the `polygon`, e.g. `milton_keynes`: it's defined as a list containing a list of two-element lists, where each of the innermost lists is a longitude-latitude pair. Also notice that the polygon is defined as a closed path, and it's closed by giving the first point in the path twice.

The solution is in the [`15.1solutions`](15.1solutions.ipynb) Notebook.

In [ ]:
milton_keynes

In [ ]:
# Insert your solution here.

### Activity 4

Colour code accidents by number of vehicles.

Find an appropriate set of bins for the number of vehicles involved in an accident (about four should do). Plot the accidents in a region with the points colour coded to show the size of the accident.

Milton Keynes has good range of accident sizes, if your own neighbourhood is safe.

The solution is in the [`15.1solutions`](15.1solutions.ipynb) Notebook.

In [ ]:
# Insert your solution here.

## What next?
If you are working through this Notebook as part of an inline exercise, return to the module materials now.

If you are working through this set of Notebooks as a whole, move on to `15.2 Aggregation pipeline`.